# Supervised classification with DeapLearning

### Importin the libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
from helpers import undersampling
from configparser import ConfigParser

In [2]:
config = ConfigParser()
config.read('config.ini')
x_size = int(config['main']['x_size'])
y_size = int(config['main']['y_size'])
clases_names = list(config['classes'].values())
nr_of_classes = len(clases_names)
csv_data_file = config['main']['csv_data_file']

### Preprocessing

In [3]:
df_unchange = pd.read_csv(csv_data_file)
#df = undersampling(df_unchange,clases_names)
df = df_unchange
X = df.iloc[:,1:-nr_of_classes].to_numpy()
Y = df.iloc[:,-nr_of_classes:].to_numpy()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

Feature scaling
$$x = \frac{\overline{x} - x}{\sigma_{x}}$$

In [5]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

TO DO Cross Validation

### Model definition

In [6]:
model = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(4, activation='softmax')
])

In [7]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Training

In [8]:
model.fit(X_train, y_train, epochs=100, batch_size=1000)

Epoch 1/100
2345/2345 [==============================] - 4s 2ms/step - loss: 0.2555 - accuracy: 0.7795
Epoch 2/100
2345/2345 [==============================] - 4s 2ms/step - loss: 0.2311 - accuracy: 0.8037
Epoch 3/100
2345/2345 [==============================] - 4s 2ms/step - loss: 0.2250 - accuracy: 0.8088
Epoch 4/100
2345/2345 [==============================] - 4s 2ms/step - loss: 0.2218 - accuracy: 0.8117
Epoch 5/100
2345/2345 [==============================] - 4s 2ms/step - loss: 0.2197 - accuracy: 0.8135
Epoch 6/100
2345/2345 [==============================] - 4s 2ms/step - loss: 0.2184 - accuracy: 0.8146
Epoch 7/100
2345/2345 [==============================] - 4s 2ms/step - loss: 0.2172 - accuracy: 0.8158
Epoch 8/100
2345/2345 [==============================] - 4s 2ms/step - loss: 0.2165 - accuracy: 0.8162
Epoch 9/100
2345/2345 [==============================] - 4s 2ms/step - loss: 0.2159 - accuracy: 0.8169
Epoch 10/100
2345/2345 [==============================] - 4s 2ms/step - l

### Validation

In [9]:
_, accuracy = model.evaluate(X_test, y_test)
print(f'acc: {accuracy}')

31396/31396 [==============================] - 37s 1ms/step - loss: 0.2135 - accuracy: 0.8178
acc: 0.8177600502967834


### Predictions

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
fig, ax = plt.subplots()
cm = cm/(0.25*X_test.shape[0])
ax.matshow(cm, cmap='seismic')
for (i, j), z in np.ndenumerate(cm):
    ax.text(j, i, '{:0.2f}'.format(z), ha='center', va='center',color = 'white')
plt.show()

### Results Presentation

In [ ]:
X_present = df_unchange.iloc[:,1:14].to_numpy()
X_present = sc.fit_transform(X_present)
Y_present = model.predict(X_present)
Y_present = np.rint(Y_present)
Y_present = Y_present.reshape((1830,1830,4))

In [ ]:
Y_target = df_unchange.iloc[:,14:].to_numpy()
Y_target = Y_target.reshape((1830,1830,4))

In [ ]:
fig, axs = plt.subplots(figsize=(12,24), ncols = 2, nrows=4)
axs[0,0].imshow(Y_present[...,0]*255,cmap='gray', vmin=0, vmax=255)
axs[0,1].imshow(Y_target[...,0]*255,cmap='gray', vmin=0, vmax=255)
axs[1,0].imshow(Y_present[...,1]*255,cmap='gray', vmin=0, vmax=255)
axs[1,1].imshow(Y_target[...,1]*255,cmap='gray', vmin=0, vmax=255)
axs[2,0].imshow(Y_present[...,2]*255,cmap='gray', vmin=0, vmax=255)
axs[2,1].imshow(Y_target[...,2]*255,cmap='gray', vmin=0, vmax=255)
axs[3,0].imshow(Y_present[...,3]*255,cmap='gray', vmin=0, vmax=255)
axs[3,1].imshow(Y_target[...,3]*255,cmap='gray', vmin=0, vmax=255)